## 대성에너지 주가 동향 Daily Report

In [23]:
import requests 
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta

ticker = '117580'   # 대성에너지

### 1. 주식 정보 가져오기

In [24]:
# fnGuide 주식 정보
url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{ticker}"
resp = requests.get(url)

if resp == 200:
    soup = BeautifulSoup(resp.text, "lxml")
    
    # 시세현황 기준일자
    selector_ref_date = '#div1 > div.um_topbar.botbd_n.bmshadow > div > div.topbar_lft > span'
    # 종가/ 전일대비
    selector_close_price = '#svdMainGrid1 > table > tbody > tr.rwf > td:nth-child(2)'
    # 52주 최고가/최저가
    selector_one_year = '#svdMainGrid1 > table > tbody > tr:nth-child(2) > td:nth-child(2)'
    # 시가총액(보통주, 억원)
    selector_market_cap = '#svdMainGrid1 > table > tbody > tr:nth-child(5) > td:nth-child(2)'
    # 거래량
    selector_volume = '#svdMainGrid1 > table > tbody > tr.rwf > td.cle.r'
    # 거래대금(억원)
    selector_tr_amt = '#svdMainGrid1 > table > tbody > tr:nth-child(2) > td.cle.r'
    # 외국인 보유비중(%)
    selector_foreigner = '#svdMainGrid1 > table > tbody > tr:nth-child(3) > td.cle.r'

    v_ref_date = soup.select_one(selector_ref_date).get_text()
    v_close_price = soup.select_one(selector_close_price).get_text()
    v_one_year = soup.select_one(selector_one_year).get_text()
    v_market_cap = soup.select_one(selector_market_cap).get_text()
    v_volume = soup.select_one(selector_volume).get_text()
    v_tr_amt = soup.select_one(selector_tr_amt).get_text()
    v_foreigner = soup.select_one(selector_foreigner).get_text()    

In [26]:
v_ref_date, v_close_price, v_one_year, v_market_cap, v_volume, v_tr_amt, v_foreigner

('[2023/07/17]',
 '73,300/ -100',
 '73,400/ 52,600',
 '4,375,851',
 ' 10,060,049',
 ' 7,368',
 '53.01')

### 2. 주식 가격 가져오기 (6개월)

In [27]:
year, month, day = v_ref_date.replace('[', '').replace(']', '').split('/')

# today = dt.datetime.now()                         # Today
# date_to = today.strftime("%Y%m%d")
# past_date = today + relativedelta(months=-6)      # 6개월 전

ref_date = year + month + day                           # 기준일
dt_ref_date = datetime.strptime(ref_date, "%Y%m%d")
dt_past_date = dt_ref_date + relativedelta(months=-6)    # 6개월 전
date_from = dt_past_date.strftime('%Y%m%d')
date_to = ref_date

In [28]:
date_from, date_to

('20230117', '20230717')

In [29]:
from pykrx import stock

# 해당 종목의 지정한 기간의 주식 거래 현황
df_period_price = stock.get_market_ohlcv(date_from, date_to, ticker) 

In [32]:
df_period_price.shape

(123, 7)

In [30]:
df_recent_price = df_period_price.tail(5).sort_values('날짜', ascending=False).reset_index()

In [31]:
df_recent_price

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,2023-07-17,7750,7900,7730,7800,50242,391124830,0.13
1,2023-07-14,7870,7870,7740,7790,42639,331877980,-0.64
2,2023-07-13,7830,7940,7830,7840,46989,369399500,0.13
3,2023-07-12,7860,7950,7780,7830,35778,280653850,0.64
4,2023-07-11,7750,7860,7750,7780,38975,303867390,0.00


### 3. 엑셀로 저장